In [24]:
import gzip, numpy, torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [13]:
batch_size = 5
nb_epochs = 100
eta = 0.01

In [14]:
((data_train, label_train), (data_test, label_test)) = torch.load(gzip.open('mnist.pkl.gz'))

C:\Users\DELL\AppData\Local\Temp\ipykernel_25792\1883767992.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ((data_train, label_train), (data_test, label_test)) = torch.

In [15]:
data_train, data_validation, label_train, label_validation = train_test_split(data_train, label_train, test_size=0.2)

In [16]:
train_dataset = torch.utils.data.TensorDataset(data_train,label_train)
test_dataset = torch.utils.data.TensorDataset(data_test,label_test)
valid_dataset = torch.utils.data.TensorDataset(data_validation,label_validation)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=1, shuffle=False)

In [17]:
class ShallowNetwork(nn.Module):
    
    def __init__(self,input_size, hidden_size, output_size):
        super(ShallowNetwork,self).__init__()
        self.hidden = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(hidden_size,output_size)
        
    def forward(self,x):
        out = self.hidden(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.output(out)
        return out

In [18]:
input_size = 784
hidden_size = 150
output_size = 10

In [19]:
model = ShallowNetwork(input_size, hidden_size, output_size)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=eta)

In [21]:
best_val_loss = float('inf')
p = 5  
t = 0

In [23]:
for epoch in range(nb_epochs):
    model.train()
    running_loss = 0
    for images ,labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss=criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    print(f"Époch {epoch+1}/{nb_epochs}, Loss = {loss.item():.4f}")
    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    val_loss /= len(valid_loader)
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")


    if val_loss < best_val_loss:
        best_val_loss = val_loss
        t = 0
    else:
        t += 1
    
    if t >= p:
        print("Early stopping!")
        break
    
    
    

Époch 1/100, Loss = 0.0776
Epoch 1, Validation Loss: 0.1944
Époch 2/100, Loss = 0.0882
Epoch 2, Validation Loss: 0.1682
Époch 3/100, Loss = 0.2174
Epoch 3, Validation Loss: 0.1435
Époch 4/100, Loss = 0.0662
Epoch 4, Validation Loss: 0.1303
Époch 5/100, Loss = 0.0334
Epoch 5, Validation Loss: 0.1232
Époch 6/100, Loss = 0.0286
Epoch 6, Validation Loss: 0.1134
Époch 7/100, Loss = 0.0219
Epoch 7, Validation Loss: 0.1064
Époch 8/100, Loss = 0.1325
Epoch 8, Validation Loss: 0.1003
Époch 9/100, Loss = 0.5749
Epoch 9, Validation Loss: 0.1006
Époch 10/100, Loss = 0.0065
Epoch 10, Validation Loss: 0.0996
Époch 11/100, Loss = 0.0175
Epoch 11, Validation Loss: 0.0942
Époch 12/100, Loss = 0.0024
Epoch 12, Validation Loss: 0.0935
Époch 13/100, Loss = 0.0103
Epoch 13, Validation Loss: 0.0925
Époch 14/100, Loss = 0.0014
Epoch 14, Validation Loss: 0.0914
Époch 15/100, Loss = 0.0369
Epoch 15, Validation Loss: 0.0899
Époch 16/100, Loss = 0.1829
Epoch 16, Validation Loss: 0.0853
Époch 17/100, Loss = 0.000

In [36]:
# Mode évaluation
model.eval()
acc=0
all_labels = []
all_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        label_multiclas = torch.argmax(labels, dim=1)
        all_labels.extend(label_multiclas.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

In [37]:
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

In [38]:
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Test Accuracy: 97.96%
Precision: 0.98
Recall: 0.98
F1-Score: 0.98
